## Process to import Boris Bike Trip Data

Data has been downloaded from the [Transport for London TFL Open Data Website](https://tfl.gov.uk/info-for/open-data-users/) into a folder called `bike_data` from where I will import, process and consolidate the data into a single Pandas dataframe.

**Import Libraries**

In [1]:
import pandas as pd
import MySQLdb
import numpy as np

**Get file names of downloaded Trip Data**

In [2]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('bike_data'):
    f.extend(filenames)
    break

**Deal with a few rogue files**

The 4 files referenced below have 3 extra columns at the end and a couple of them have a load of blank records at the bottom that I'm excluding by manually setting the number of rows to import.

**Loop through files, import the csv and add to master df**

In [3]:
counter = 0
for file in f:
    counter += 1
    try:
        x = pd.read_csv('bike_data/' + file)
        x = x.dropna()
        master = pd.concat([master, x], axis = 0)
        print file + ' ' + str(counter) + ' of ' + str(len(f))
    except:
        master = pd.read_csv('bike_data/' + file)

12aJourneyDataExtract15Nov15-27Nov15.csv 2 of 79
2. Journey Data Extract 03Feb14-01Mar14.csv 3 of 79
13a. Journey Data Extract 07Dec14-21Dec14.csv 4 of 79
8. Journey Data Extract 23Jun-20Jul13.csv 5 of 79


/home/jb/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


6. Journey Data Extract_27May-23Jun12.csv 6 of 79
7. Journey Data Extract_24Jun-21Jul12.csv 7 of 79
02bJourneyDataExtract21Feb16-05Mar2016.csv 8 of 79
11b. Journey Data Extract 12Oct14-08Nov14.csv 9 of 79
8b-Journey-Data-Extract-08Aug15-22Aug15.csv 10 of 79
11a-Journey-Data-Extract-18Oct15-31Oct15.csv 11 of 79
13bJourneyDataExtract25Dec15-09Jan16.csv 12 of 79
5b. Journey Data Extract 17May15-30May15.csv 13 of 79
6. Journey Data Extract 25May14-21Jun14.csv 14 of 79
5. Journey Data Extract 27Apr14-24May14.csv 15 of 79
2. Journey Data Extract_01Feb-29Feb 12.csv 16 of 79
4a. Journey Data Extract 01Apr15-16Apr15.csv 17 of 79
6. Journey Data Extract 28Apr-25May13.csv 18 of 79
9b-Journey-Data-Extract-06Sep15-19Sep15.csv 19 of 79
3a. Journey Data Extract 01Mar15-15Mar15.csv 20 of 79
1. Journey Data Extract 04Jan-31Jan 12.csv 21 of 79
8b Journey Data Extract 01Aug14-16Aug14.csv 22 of 79
11. Journey Data Extract 14Sep13-12Oct13.csv 23 of 79
5. Journey Data Extract_29Apr-26May12.csv 24 of 79
2. J

/home/jb/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


9a Journey Data Extract 17Aug14-31Aug14.csv 48 of 79
02aJourneyDataExtract07Fe16-20Feb2016.csv 49 of 79
17. Journey Data Extract 01Nov-30Nov12.csv 50 of 79
7b. Journey Data Extract 12Jul15-25Jul15.csv 51 of 79
7. Journey Data Extract 22Jun14-19Jul14.csv 52 of 79
16. Journey Data Extract 01Oct-31Oct12.csv 53 of 79
13aJourneyDataExtract13Dec15-24Dec15.csv 54 of 79
3. Journey Data Extract_01Mar-31Mar12.csv 55 of 79
12b. Journey Data Extract 09Nov14-06Dec14.csv 56 of 79
18. Journey Data Extract 01Dec-31Dec12.csv 57 of 79
15. Journey Data Extract 01Sep-30Sep12.csv 58 of 79
14. Journey Data Extract 08Dec13-04Jan14.csv 59 of 79
7a. Journey Data Extract 28Jun15-11Jul15.csv 60 of 79
10. Journey Data Extract 21Aug-22 Aug12.csv 61 of 79
10a-Journey-Data-Extract-20Sep15-03Oct15.csv 62 of 79
8a-Journey-Data-Extract-26Jul15-07Aug15.csv 63 of 79
6a Journey Data Extract 31May15-12Jun15.csv 64 of 79
10a. Journey Data Extract 14Sep14-27Sep14.csv 65 of 79
5a. Journey Data Extract 03May15-16May15.csv 66 o

**Quick Inspection of the Data**

In [4]:
print 'Data Frame of ' + str(master.shape[0]) + ' rows and ' + str(master.shape[1]) + ' columns: \n'
print(master.dtypes)

Data Frame of 37267718 rows and 9 columns: 

Rental Id            float64
Duration             float64
Bike Id              float64
End Date              object
EndStation Id        float64
EndStation Name       object
Start Date            object
StartStation Id       object
StartStation Name     object
dtype: object


In [5]:
master.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,41215296,120,12077,15/02/2015 00:02,647,"Richmond Way, Shepherd's Bush",15/02/2015 00:00,613,"Woodstock Grove, Shepherd's Bush"
1,41215294,120,7892,15/02/2015 00:02,647,"Richmond Way, Shepherd's Bush",15/02/2015 00:00,613,"Woodstock Grove, Shepherd's Bush"
2,41215295,1140,3903,15/02/2015 00:19,469,"Lindfield Street, Poplar",15/02/2015 00:00,448,"Fishermans Walk West, Canary Wharf"
3,41215303,720,5719,15/02/2015 00:13,487,"Canton Street, Poplar",15/02/2015 00:01,200,"LMU Commercial Road, Whitechapel"
4,41215298,180,3695,15/02/2015 00:04,321,"Bermondsey Street, Bermondsey",15/02/2015 00:01,269,"Empire Square, The Borough"


<br>
**Inital conclusions on the Data **

_Problems_
* Start and End date need to be converted to datetime (tried this on import, it was crazy slow).
* That `StartStation Id` is an object is suspcious as they appear to be numerical.
* A quick group by on the Start/End Station Names tells us that there is a fair but of inconsistency amongst them over time.
* There's also a few mangled records with strange durations and start times.

In [6]:
master.loc[master['EndStation Name'] == '6,0'].head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
20783,12542513,749724,8999,05/06/2012 12:06:16,434,"6,0",22/03/1901 00:00:00,200054,"Cubitt Town,0"
46980,12907812,731501,8731,18/06/2012 11:05:46,434,"6,0",20/07/1900 00:00:00,1102,"Aldgate,0"
120353,13207448,246794,7981,23/06/2012 09:28:10,434,"6,0",03/03/1901 00:00:00,22179,"South Kensington,0"
180334,12593058,573308,7352,05/06/2012 11:15:16,434,"6,0",15/02/1900 00:00:00,1031,"Wapping,0"
222113,12767871,335562,6938,07/06/2012 11:50:00,434,"6,0",01/12/1900 00:00:00,1212,"South Bank,1"


**Conclusion**

Records with an EndStation Name of `6,0` seem to be mangled - the good news is that there aren't many of them so I will simply exclude. 

The non numeric start id is caused by a handful of records with the value 'Tabletop1' which I will also exclude.

Duration also seems to have been rounded in the later files rendering the field completely useless, I will calculate it manually.

In [7]:
#remove the '6,0' records
master = master[master['EndStation Name'] != '6,0']

In [8]:
#select relevant columns and improve names
data = master[['Rental Id', 'Bike Id', 'StartStation Id', 'Start Date', 'EndStation Id', 'End Date']]
data.columns = ['rental_id', 'bike_id', 'start_id', 'start_time', 'end_id', 'end_time']

In [9]:
data.head()

,rental_id,bike_id,start_id,start_time,end_id,end_time
0,41215296,12077,613,15/02/2015 00:00,647,15/02/2015 00:02
1,41215294,7892,613,15/02/2015 00:00,647,15/02/2015 00:02
2,41215295,3903,448,15/02/2015 00:00,469,15/02/2015 00:19
3,41215303,5719,200,15/02/2015 00:01,487,15/02/2015 00:13
4,41215298,3695,269,15/02/2015 00:01,321,15/02/2015 00:04


**Get rid of the weird tabletop records**

In [10]:
data = data[data['start_id'] != 'Tabletop1']

**Convert columns to appropriate data types**

In [ ]:
data['start_id'] = pd.to_numeric(data['start_id'], errors='ignore')
data['start_time'] = pd.to_datetime(data['start_time'], dayfirst = True)
data['end_time'] = pd.to_datetime(data['end_time'], dayfirst = True)

In [ ]:
data.dtypes

In [14]:
data.groupby(data.start_time.dt.date).count()['rental_id']

start_time
1900-01-02        1
1900-01-04        2
1900-01-05        1
1900-01-08        1
1900-01-09        3
1900-01-12        1
1900-01-14        6
1900-01-15        1
1900-01-16        1
1900-01-17        2
1900-01-18        5
1900-01-22        1
1900-01-23        1
1900-01-24        1
1900-01-25        1
1900-01-27        1
1900-01-28        2
1900-01-29        4
1900-01-31        2
1900-02-01        5
1900-02-02        6
1900-02-03        1
1900-02-05        2
1900-02-06        3
1900-02-07        1
1900-02-08        9
1900-02-09        3
1900-02-10        3
1900-02-12        3
1900-02-13        4
              ...  
2016-01-08    20761
2016-01-09    11583
2016-02-07    14101
2016-02-08    19850
2016-02-09    23306
2016-02-10    23626
2016-02-11    25184
2016-02-12    22819
2016-02-13     9868
2016-02-14    13697
2016-02-15    21787
2016-02-16    24386
2016-02-17    19033
2016-02-18    23516
2016-02-19    22172
2016-02-20    11135
2016-02-21    14669
2016-02-22    20463
2016-02-2

**'Pickle' dataframe for future use**

In [11]:
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    Source: http://stackoverflow.com/questions/29882573/pandas-slow-date-conversion
    """
    dates = {date:pd.to_datetime(date, dayfirst = True) for date in s.unique()}
    return s.apply(lambda v: dates[v])

In [12]:
data['start_time'] = lookup(data['start_time'])
data['end_time'] = lookup(data['end_time'])
data = data.dropna()

In [13]:
data.dtypes

rental_id            float64
bike_id              float64
start_id              object
start_time    datetime64[ns]
end_id               float64
end_time      datetime64[ns]
dtype: object

In [15]:
data.to_pickle('consol_data')